1. 準備（ライブラリのインストール）
2. データセットの準備
3. データの前処理（BERTが受けとれる形式にデータを整形）
4. 学習済みモデルのロード
5. 訓練（Pre-trainedモデルのファインチューニング）
6. 検証

#0. 作業環境の準備

In [2]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


In [3]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#1. 準備

Hugging Face Library 'Transformer' のダウンロード

* transformers
* 最新のNLPモデルを利用できるライブラリ
* BERTの他にもGPT-2, RoBERTa, XLM, DistilBert, XLNet, T5, CTRL...などの最新モデルや100言語以上の学習済みモデル（Pre-trained model）を利用可能

In [4]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 45.0 MB/s 
     |████████████████████████████████| 895 kB 33.7 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 3.3 MB 36.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-do

#2. データセットの準備

##2.1.ライブドアニュースコーパスをダウンロードする。

ダウンロードしたファイルは圧縮（tar.gz形式）ファイルです

解凍すると様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダが分かれており、それぞれの記事がテキストファイル形式で保存されています

データセットの準備はライブドアニュースコーパスのダウンロードを参考にさせて頂きました

In [5]:
# import os
# import urllib.request
# import re
# import csv
# import tarfile

import pandas as pd
df_v5=pd.read_csv('/content/drive/MyDrive/研究室/研究_M2/src/Assets/Assets_Output/Tweet_Datasets/v5_forBERT.csv', engine='python')



---



##2.2. 必要なデータ（文章とラベル）だけ抽出します

In [6]:
import pandas as pd

# データの読み込み
#df = pd.read_csv("all_text.tsv", delimiter='\t', header=None, names=['media_name', 'label_EorI', 'NaN', 'sentence'])
df = df_v5
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (483083, 16)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,author_id,text,tweet_id,tweet_time,conversation_id,lang,MBTI_Type,notes,label_EorI,label_NorS,label_ForT,label_JorP
468628,869,74,74,74,3.097728e+09,@kiichimpanzee おい！,9.190753e+17,2017-10-14 05:40:12+00:00,9.190269e+17,ja,ESFP,NaN,E,S,F,P
161466,20162,90,90,90,2.803834e+09,りんなって留年繰り返してるんやろか…,9.183864e+17,2017-10-12 08:02:28+00:00,9.183864e+17,ja,INFP,NaN,I,N,F,P
396560,30318,52,52,52,2.910324e+09,@katosato_FH @shinobu_0210 それは否定する意見だけでも同じじゃない...,9.267545e+17,2017-11-04 10:14:33+00:00,9.267469e+17,ja,ISTP,NaN,I,S,T,P
25029,2547,10,10,10,2.450622e+09,RT @gamepedia_alert: スプラトゥーン2のamiiboがセブンネットショッ...,9.445650e+17,2017-12-23 13:46:55+00:00,9.445650e+17,ja,INTP,NaN,I,N,T,P
390858,24616,87,87,87,3.882458e+08,やってるのはフリーのホラーゲームなんだけどな！！\nいちゃついてるホラゲってなんだよ！！！,9.116546e+17,2017-09-23 18:12:55+00:00,9.116546e+17,ja,ISTP,NaN,I,S,T,P
374701,8459,60,60,60,6.034139e+08,聖なる夜勤が始まった（3年連続3回目）,9.449205e+17,2017-12-24 13:19:45+00:00,9.449205e+17,ja,ISTP,NaN,I,S,T,P
235910,94606,82,82,82,2.472378e+09,ありがとう、優しいお姉さん方。無事譲治さん見れます。,9.168465e+17,2017-10-08 02:03:33+00:00,9.168465e+17,ja,INFP,NaN,I,N,F,P
478651,10892,46,46,46,2.905124e+09,RT @PKUZNchamcham: こうやってカイテムにソンウンさんお呼ばれしたのね☺️💓...,9.378316e+17,2017-12-04 23:50:40+00:00,9.378316e+17,ja,ESFP,NaN,E,S,F,P
313029,16035,23,23,23,1.269421e+08,@rin_0426 当日はお誕生日お祝い頂きありがとうございました！また、謎とか食事とか、ご...,9.460436e+17,2017-12-27 15:42:29+00:00,9.460436e+17,ja,ISFJ,NaN,I,S,F,J
95337,13514,49,49,49,1.860174e+08,返す刀で学生のジャーナル論文をチェックする．,9.451178e+17,2017-12-25 02:23:45+00:00,9.451178e+17,ja,ENTP,NaN,E,N,T,P


In [7]:
df=df.drop('notes', axis=1)
df=df.dropna(how='any', axis=0)

In [8]:
df_label=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df_label

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,author_id,text,tweet_id,tweet_time,conversation_id,lang,MBTI_Type,label_EorI_E,label_EorI_I,label_NorS_N,label_NorS_S,label_ForT_F,label_ForT_T,label_JorP_J,label_JorP_P
0,0,0,0,0,8.726092e+07,RT @det_kirigakure: #毎日龍驤\n謹賀新年、明けましておめでとうございま...,9.474884e+17,2017-12-31 15:23:39+00:00,9.474884e+17,ja,INTJ,0,1,1,0,0,1,1,0
1,1,1,1,1,8.726092e+07,RT @nunnu_zero: 今流行りの壺をうちの子にプレゼントしてみた https://...,9.474178e+17,2017-12-31 10:42:48+00:00,9.474178e+17,ja,INTJ,0,1,1,0,0,1,1,0
2,2,2,2,2,8.726092e+07,RT @wahiko94: 下着姿の加賀殿を描いたでござる https://t.co/rX2...,9.474175e+17,2017-12-31 10:41:44+00:00,9.474175e+17,ja,INTJ,0,1,1,0,0,1,1,0
3,3,3,3,3,8.726092e+07,RT @whimgreen: ユニコーンちゃんに膝枕してもらいたい人生だった #アズールレー...,9.471055e+17,2017-12-30 14:01:55+00:00,9.471055e+17,ja,INTJ,0,1,1,0,0,1,1,0
4,4,4,4,4,8.726092e+07,RT @whimgreen: スカートの中見せてくれるのかと期待して「すけべ♪」って言われた...,9.471054e+17,2017-12-30 14:01:38+00:00,9.471054e+17,ja,INTJ,0,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483078,15319,90,90,90,3.885770e+09,もう無理なんでそんな元気なんこれから仕事行って第2ラウンドカラオケとかもうどうなってるんその体,8.969414e+17,2017-08-14 03:47:40+00:00,8.969414e+17,ja,ESFP,1,0,0,1,1,0,0,1
483079,15320,91,91,91,3.885770e+09,こんな時間から声なんて出ないよあほむすめぇぇぇぇ\nｹﾞﾎｯ,8.968560e+17,2017-08-13 22:08:39+00:00,8.968560e+17,ja,ESFP,1,0,0,1,1,0,0,1
483080,15321,92,92,92,3.885770e+09,コンタクトどこ行った？\n→私「まさか捨てた？」\n→捨ててたわ\n\nていうかマンガみたい...,8.968342e+17,2017-08-13 20:42:00+00:00,8.968342e+17,ja,ESFP,1,0,0,1,1,0,0,1
483081,15322,93,93,93,3.885770e+09,お薬スープ200、角煮100、お寿司100、\n……仕込みが終わった。これでせめて１日位もっ...,8.957653e+17,2017-08-10 21:54:24+00:00,8.957653e+17,ja,ESFP,1,0,0,1,1,0,0,1


##2.3. 文章データをsentences、ラベルデータを labelsという変数に保存します

次パートからはこの2変数だけを利用します

In [9]:
# データの抽出
sentences = df.text.values #sentences = df.sentence.values
labels = df_label.label_EorI_E.values #labels = df.label.values

#3. データの前処理（BERTが受けとれる形式にデータを整形）

BERTでは基本的に学習済みモデルを利用する為、そのモデルが読み込めるフォーマットにデータを変換する必要があります
具体的には、以下の4つ手続きが必要になります

1. BERT Tokenizerを用いて単語分割・IDへ変換
>* 学習済みモデルの作成時と同じtokenizer（形態素解析器）を利用する必要がある
>* 日本語ではMecabやJuman++を利用されることが多い

2. Special tokenの追加
>* 文章の最後に[SEP]という単語する
>* 文章のはじめに[CLS]という単語を追加する（分類問題に利用される）
3. 文章の長さの固定
>* BERTでは全ての文書の長さ（単語の数）を同じにする必要がある（1文章あたりの最大の単語数は512単語）
>* そこで、Padding/Truncatingを用いて固定長に変換する
>* Paddingとは、指定した長さに満たない文章を[Pad]という意味を持たない単語の埋める処理
>* Truncatingとは、指定した長さを超える単語を切り捨てること
4. Attention mask arrayの作成
>* [Padding]を0、それ以外のTokenを1とした配列
>* 一見ややこししそう見えますが、どれもTransformerのライブラリでサポートされているので、簡単に実行できます

また、以下のイメージも参考にモデルの全体像と照らし合わせながら前処理内容をみるとイメージが沸きやすいかもしれません

##3.1. BERT Tokenizerを用いて単語分割・IDへ変換

In [10]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


#念の為、確認
## テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

 Original:  RT @det_kirigakure: #毎日龍驤
謹賀新年、明けましておめでとうございます！！今年もよろしくお願いします！！皆様にとって良い年になりますように！！やで！！(*'ω'*) https://t.co/voYpVuuBiE
Tokenized:  ['RT', '@', 'de', '##t', '_', 'k', '##ir', '##iga', '##k', '##ure', ':', '#', '毎日', '龍', '[UNK]', '謹', '##賀', '新年', '、', '明け', 'まし', 'て', 'お', '##め', '##で', '##とう', 'ござい', 'ます', '!!', '今年', 'も', 'よろ', '##しく', 'お願い', 'し', 'ます', '!!', '皆', '##様', 'にとって', '良い', '年', 'に', 'なり', 'ます', 'よう', 'に', '!!', 'や', 'で', '!!', '##(', '##*', "##'", '[UNK]', "'", '##*', '##)', 'http', '##s', '://', 't', '.', 'co', '/', 'v', '##o', '##Y', '##p', '##V', '##u', '##u', '##B', '##i', '##E']
Token IDs:  [17232, 13817, 3560, 28566, 1679, 613, 2366, 27709, 28807, 5975, 266, 4931, 3904, 2423, 1, 19060, 29404, 24779, 6, 9073, 3913, 16, 73, 28506, 28455, 3671, 27378, 2610, 3421, 19241, 28, 23223, 838, 24050, 15, 2610, 3421, 5257, 28857, 2032, 3614, 19, 7, 297, 2610, 124, 7, 3421, 49, 12, 3421, 28471, 30522, 29482, 1, 1507, 30522, 28473, 21313, 28589, 16831, 822, 143, 20852,

##3.2. 次に文章の長さを固定する為に、文章あたりの最大単語数を確認します

ここでは、以下の方法で最大単語数を確認していますが、最大単語数は決め打ちで設定することも可能です

In [11]:
print(sentences[15806])

@anju_u102dn ﾎﾟｲﾝﾄｶﾞﾁｬだし演出スキップしてしまった、悲しい


In [12]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)
for idx, sent in enumerate(sentences):
    print(idx, len_sentences)
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

ストリーミング出力は最後の 5000 行に切り捨てられました。
477186 482184
477187 482184
477188 482184
477189 482184
477190 482184
477191 482184
477192 482184
477193 482184
477194 482184
477195 482184
477196 482184
477197 482184
477198 482184
477199 482184
477200 482184
477201 482184
477202 482184
477203 482184
477204 482184
477205 482184
477206 482184
477207 482184
477208 482184
477209 482184
477210 482184
477211 482184
477212 482184
477213 482184
477214 482184
477215 482184
477216 482184
477217 482184
477218 482184
477219 482184
477220 482184
477221 482184
477222 482184
477223 482184
477224 482184
477225 482184
477226 482184
477227 482184
477228 482184
477229 482184
477230 482184
477231 482184
477232 482184
477233 482184
477234 482184
477235 482184
477236 482184
477237 482184
477238 482184
477239 482184
477240 482184
477241 482184
477242 482184
477243 482184
477244 482184
477245 482184
477246 482184
477247 482184
477248 482184
477249 482184
477250 482184
477251 482184
477252 482184
477253 482184
477254 482184
47

##3.3. Tokenizerと最大単語数の確認がとれたので、全ての文章に一括して処理を行います

tokenizer.encode_plusを利用するとまとめて処理が行えます

In [14]:
from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 37,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

  0%|          | 0/482184 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  RT @det_kirigakure: #毎日龍驤
謹賀新年、明けましておめでとうございます！！今年もよろしくお願いします！！皆様にとって良い年になりますように！！やで！！(*'ω'*) https://t.co/voYpVuuBiE
Token IDs: tensor([    2, 17232, 13817,  3560, 28566,  1679,   613,  2366, 27709, 28807,
         5975,   266,  4931,  3904,  2423,     1, 19060, 29404, 24779,     6,
         9073,  3913,    16,    73, 28506, 28455,  3671, 27378,  2610,  3421,
        19241,    28, 23223,   838, 24050,    15,     3])


##labelをtensor型に変換

labelはpd.get_dummiesでone-hotにしてます

In [15]:
# tenosor型に変換
labels = torch.tensor(labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


## BERTの前処理が正常に実行されたか確認

前処理は、

Special tokenの追加、文章の長さの固定、Attention mask arrayの作成

である。

In [16]:
# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  RT @det_kirigakure: #毎日龍驤
謹賀新年、明けましておめでとうございます！！今年もよろしくお願いします！！皆様にとって良い年になりますように！！やで！！(*'ω'*) https://t.co/voYpVuuBiE
Token IDs: tensor([    2, 17232, 13817,  3560, 28566,  1679,   613,  2366, 27709, 28807,
         5975,   266,  4931,  3904,  2423,     1, 19060, 29404, 24779,     6,
         9073,  3913,    16,    73, 28506, 28455,  3671, 27378,  2610,  3421,
        19241,    28, 23223,   838, 24050,    15,     3])


1つ目の文章の前処理結果です

元々、日本語テキストだった文章がID化されています

Token IDsの最初に単語IDはSpecial tokenの[CLS]を表し、後半の0埋めが[Pad]を示しています



```
Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Token IDs: tensor([    2, 18718,  8653,    12,  1763, 29135,     5, 20734,   679,  6215,
         3794,    49,  1604,    11, 24598,     7,  9398,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
```



##3.5. データローダの適用とデータの分割

ここまでで、前処理が完了したので、90%を訓練データ、10％をテストデータに分割して、pytorchで学習を行う為に、データローダーへ変換しておきます

データローダーの説明は、pytorchの基本的な操作なので本記事では割愛します（簡単に言うと、データをバッチごとに分割して、学習を上手くやってくれるデータ型です）

In [17]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：433965
検証データ数:　48219 


#4. 学習済みモデルのロード

huggingface transformerのBERTファインチューニングでは以下のタスクをサポートしています
全てBERT Pre-trainedモデルをベースとして学習し、出力層のみそれぞれのタスクに適した構成になっています

* BertModel
* BertForPreTraining
*   BertForMaskedLM
*   BertForNextSentencePrediction
*   BertForSequenceClassification
*   BertForTokenClassification
* BertForQuestionAnswering

公式ドキュメント.

## BertForSequenceClassification

今回は2クラス分類問題なのでBertForSequenceClassificationを利用します

BertForSequenceClassificationは**「BERT学習済みモデルの最後の層に分類用のレイヤー追加したネットワーク構成」**です

データを入力することで、学習済みモデル全体と未学習部分の分類機レイヤーの学習が行われます

デフォルトでは、モデル全体のパラメーターがファインチューニングされますが、学習させるレイヤーの設定変更も可能です

モデルロード時にnum_labelsを変更することで多クラス分類問題へも対応できます

In [18]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

学習済みモデルのロードが完了しました

次にこのモデルを手元のデータに合うようにファインチューニングを行います

#5. 訓練（Pre-trainedモデルのファインチューニング）

モデルのファインチューニングは通常のニューラルネットワークの学習と同様で、損失関数と最適化手法（Optimizer）を指定して、学習ループ（任意のバッチサイズとエポックを指定）を回します

BERTでは損失関数はすでに定義されているので、ハイパーパラメーターとして、最適化手法とバッチサイズをエポック数だけ指定します

ここでは、論文（BERT paper, Appendix A.3）に従って、最適化関数はAdamW（Adam Weight Decay fix）を利用し、ハイパーパラメーターは下記のように設定します

Batch size: 32
Learning rate: 2e-5
Epochs: 4

In [19]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        logits= outputs.logits
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                    outputs = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            loss = outputs.loss
            logits= outputs.logits
            val_loss += loss.item()
    return val_loss

学習に必要な関数が定義できたので、学習を行う

In [1]:
# 学習の実行
max_epoch = 2
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)

    # 学習途中の状態を保存する。
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
         },
         "model.tar",
    )
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

NameError: ignored

##5.2. モデルの保存

https://pystyle.info/pytorch-save-and-load-model/

In [ ]:
# 学習途中の状態を保存する。
torch.save(
    {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
    },
    "model.tar",
)

# 学習途中の状態を読み込む。
# checkpoint = torch.load("model.tar")
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# epoch = checkpoint["epoch"]

#6. 検証

In [ ]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

以下の様な結果が出力されます
この確率の様な値（厳密には、この値ををソフトマックス関数に入力すると確率になる）が大きい方のラベルをモデルは分類結果とし予測としています

In [ ]:
## 予測結果の確認
print(f'出力:{preds}')



```
出力:(tensor([[-5.0226,  5.0193],
        [-5.0390,  4.9736],
        [ 4.7941, -4.7459],
        [-4.9395,  4.6827], device='cuda:0'),)
```



左からラベルの[0, 1]に対応しており、1つ目のデータであれば、右側の値が大きいのでモデルはラベル1を予測している

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

上記の様に予測ラベルと正解ラベルを取得して、dataframeへ保存することができました

あとは、シンプルに正解率を計算するなり、sklearnを用いて混合行列を作成するなり、好きな様に分類モデルの評価を行うことができます